In [23]:
import hashlib
import time

class Block:
    """
    Represents a single block in the blockchain.
    Each block contains an index, timestamp, data, proof, its own hash, and the hash of the previous block to maintain linkage.
    """
    def __init__(self, index, previous_hash, timestamp, data, proof):
        self.index = index # Block number in the chain
        self.previous_hash = previous_hash #Stores the hash of the previous block in the chain
        self.timestamp = timestamp #Time the block was created
        self.data = data #Transaction or arbitrary data input
        self.proof = proof #proof of work
        self.hash = self.calculate_hash() #This block's hash value

    def calculate_hash(self):
        # Combine all key values of the block into a string
        block_string = f"{self.index}{self.previous_hash}{self.timestamp}{self.data}{self.proof}"
        # Encode the string and calculate the SHA-256 hash
        return hashlib.sha256(block_string.encode()).hexdigest()




class Blockchain:
    """
    This class represents the entire blockchain system.
    It handles all the blocks and operations like adding blocks, linking them, and checking the chain.
    """
    def __init__(self):
        self.difficulty = 4  # Number of leading zeros required in a valid hash (Proof of Work)
        self.chain = [self.create_genesis_block()] #Start with first block

    def create_genesis_block(self):
        """
        This method creates the first block in the blockchain.
        It doesn't have a real previous block, so we just use "0".
        Data is set as "Genesis Block" by default
        """
        genesis_block = Block(
            index=0,
            previous_hash="0",
            timestamp=time.time(),
            data="Genesis Block",
            proof=0
        )
        return self.proof_of_work(genesis_block)

    def get_latest_block(self):
        """
        This method returns the most recent block in the blockchain.
        It's usually used when we want to add a new block and need the latest hash.
        """
        return self.chain[-1]

    def add_block(self, new_block):
        """
        This method adds a new block to the blockchain. Before appending, it links the new block to the previous one by updating its previous_hash.
        After setting the previous-hash, it recalculates the block's own hash and then adds it to the chain.
        """

        # Add a new block to the chain by assigning the latest block's hash
        new_block.previous_hash = self.get_latest_block().hash
        # Since the previous hash changed, we must recalculate this block's hash
        new_block.hash = new_block.calculate_hash()
        # Finally, append the block to the chain
        self.chain.append(new_block)

    def is_valid_proof(self,data,proof):
        """
        Checks if the hash of the (data + proof) starts with '0000'.
        Used during mining (proof of work).
        """
        guess = str(data) + str(proof)
        guess_hash = hashlib.sha256(guess.encode()).hexdigest()
        return guess_hash.startswith("0" * self.difficulty)
        
    def proof_of_work(self, block):
        """
        Implement the proof-of-work algorithm
        Increment the proof value until the block's hash starts with the required number of leading zeros
        """
        while not block.hash.startswith("0" * self.difficulty):
            block.proof += 1 # Keep trying different proof values
            block.hash = block.calculate_hash() #Recalculate hash with updated proof

        # when the while loop ends, we found a valid proof
        return block

    def add_data(self, data):
        """
        This method lets the user add new data to the blockchain.
        It creates a new block with that data, mines it using proof_of_work, and adds it to the chain.
        """
        # Get the latest block so we know the previous hash
        previous_block = self.get_latest_block()

        # Create a new block with updated index and data
        new_block = Block(
            index=previous_block.index + 1,
            previous_hash=previous_block.hash,
            timestamp=time.time(),
            data=data,
            proof=0
        )

        # Run proof of work to get a valid hash
        mined_block = self.proof_of_work(new_block)

        # Add the mined block to the chain
        self.add_block(mined_block)



    def is_chain_valid(self):
        """
        This method checks if the entire blockchain is still valid.
        Make sure that each block's hash is correct and properly linked to the previous one.
        """
        for i in range(1,len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]

            # Recalculate the hash and compare
            if current_block.hash != current_block.calculate_hash():
                print(f"Block{current_block.index} has invalid hash.")
                return False
            
            #Check if the previous hash matches the last block's hash
            if current_block.previous_hash != previous_block.hash:
                print(f"Block{current_block.index} is not properly linked.")
                return False

        return True # If all blocks are valid


# Example Usage
if __name__ == "__main__":
    blockchain = Blockchain()

    print("Mining block 1...")
    blockchain.add_data("Transaction data for Block 1")

    print("Mining block 2...")
    blockchain.add_data("Transaction data for Block 2")

    print("\nBlockchain validity:", blockchain.is_chain_valid())

    for block in blockchain.chain:
        print(f"Block {block.index} | Hash: {block.hash} | Previous Hash: {block.previous_hash}")



Mining block 1...
Mining block 2...

Blockchain validity: True
Block 0 | Hash: 0000ed310f25a26e3f3623bface3f4682396479729466ca3e3a9059975d58262 | Previous Hash: 0
Block 1 | Hash: 0000a0c2b45bf53e6b60a47421d1da7646e2937a368bdc967ed96864846e7109 | Previous Hash: 0000ed310f25a26e3f3623bface3f4682396479729466ca3e3a9059975d58262
Block 2 | Hash: 00006360f55edab1f3f6e3dfd3ef3beb0412355c8c14a02703b9b46aa880d487 | Previous Hash: 0000a0c2b45bf53e6b60a47421d1da7646e2937a368bdc967ed96864846e7109


In [24]:

def print_blockchain(chain):
    print("\n====== Blockchain Contents ======")
    for block in chain:
        print("\n-------------------------------")
        print(f"Block Index: {block.index}")
        print(f"Timestamp  : {block.timestamp}")
        print(f"Data       : {block.data}")
        print(f"Proof      : {block.proof}")
        print(f"Hash       : {block.hash}")
        print(f"Prev Hash  : {block.previous_hash}")

# Blockchain Menu
if __name__ == "__main__":
    blockchain = Blockchain()

    while True:
        print("\n==== Blockchain Menu ====")
        print("1. Add a new block")
        print("2. Display the blockchain")
        print("3. Validate the blockchain")
        print("4. Exit")
        
        choice = input("Enter your choice(1-4): ")

        if choice == "1":
            data = input("Enter transaction data: ")
            print("Mining block... please wait.")
            blockchain.add_data(data)
            print("Block added successfully.")
        
        elif choice == "2":
           print_blockchain(blockchain.chain)

        elif choice == "3":
            if blockchain.is_chain_valid():
                print("Blockchain is VALID.")
            else:
                print("Blockchain is INVALID!")
                
        elif choice == "4":
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please enter 1-4.")


==== Blockchain Menu ====
1. Add a new block
2. Display the blockchain
3. Validate the blockchain
4. Exit
Mining block... please wait.
Block added successfully.

==== Blockchain Menu ====
1. Add a new block
2. Display the blockchain
3. Validate the blockchain
4. Exit
Mining block... please wait.
Block added successfully.

==== Blockchain Menu ====
1. Add a new block
2. Display the blockchain
3. Validate the blockchain
4. Exit

====== Blockchain Contents ======

-------------------------------
Block Index: 0
Timestamp  : 1752934618.95385
Data       : Genesis Block
Proof      : 31615
Hash       : 00007475552df665669b0540a9e3633b9c015fa13296e7784e9a28122ee94381
Prev Hash  : 0

-------------------------------
Block Index: 1
Timestamp  : 1752934628.226769
Data       : saba
Proof      : 27871
Hash       : 0000192529152c7afd93c1c6d98f8fda573a8ecebb2fb8b1316ba0ae357aa20e
Prev Hash  : 00007475552df665669b0540a9e3633b9c015fa13296e7784e9a28122ee94381

-------------------------------
Block Index: